# File Consolidation

In [9]:
# Import libraries

import pandas as pd
import numpy as np
import os
import glob
import json

from tqdm import tqdm

from collections import defaultdict

In [10]:
# Navigate to file folder and retrieve relevant files
all_files = glob.glob(r"C:\Users\jawo19ad\Dropbox (CBS)\Master thesis data\Twitter Fetch\*.txt")

In [11]:
def load_full_json(file):
    
    # Create empty dictionary where values are stored
    json_complete = {"data":list(),"users":list()}
    
    # Open the json file
    with open(file, "r") as json_file:
        
        # Iterate through json file and store information at appropriate dict key
        for line in json_file:
            data = json.loads(line)
            json_complete["data"].extend(data["data"])
            json_complete["users"].extend(data["includes"]["users"])
    
    return json_complete

In [ ]:
json_complete = {"data":list(),"users":list()}
tweets_complete = {"data":list()}
users_complete = {"users":list()}


for file in tqdm(all_files):
    json_file = load_full_json(file)
    #print(len(json_file["data"]), len(json_file["users"]))
    tweets_complete["data"].extend(json_file["data"])
    users_complete["users"].extend(json_file["users"])

 54%|█████▍    | 14/26 [08:13<06:14, 31.24s/it]

In [ ]:
len(tweets_complete["data"])

In [ ]:
len(users_complete["users"])

In [ ]:
# Store tweets file
with open(r'C:\Users\jawo19ad\Dropbox (CBS)\Master thesis data\all_tweets_012020_042021.json', 'w') as f:
    json.dump(tweets_complete, f)

In [ ]:
# Store users file
with open(r'C:\Users\jawo19ad\Dropbox (CBS)\Master thesis data\all_users_012020_042021.json', 'w') as f:
    json.dump(users_complete, f)

# Creation of df_user

In [ ]:
# Load the json file containing the users
with open(r'C:\Users\jawo19ad\Dropbox (CBS)\Master thesis data\all_users_032020_042021.json', 'r') as f:
    data = json.load(f)

In [ ]:
# Convert json file into pandas dataframe
df_users = pd.json_normalize(data["users"])
df_users.head()

In [ ]:
df_users.shape

In [ ]:
# Remove duplicate users
df_users_no_dups = df_users.drop_duplicates(subset="id")

In [ ]:
df_users_no_dups.shape

In [ ]:
# Drop irrelevant columns
df_users_no_dups.drop(['entities.description.hashtags','entities.url.urls', 'entities.description.urls',
                       'entities.description.mentions', 'entities.description.cashtags', 'withheld.country_codes'],
                       axis=1, inplace=True)

In [ ]:
# Store file in csv format
df_users_no_dups.to_csv(r"C:\Users\jawo19ad\Dropbox (CBS)\Master thesis data\df_users.csv")

# Creation of df_tweets

In [ ]:
# Load the json file containing the tweets
with open(r'C:\Users\jawo19ad\Dropbox (CBS)\Master thesis data\all_tweets_032020_042021.json', 'r') as f:
    data = json.load(f)

In [ ]:
# Convert json file into pandas dataframe
df_tweets = pd.json_normalize(data["data"])
df_tweets.head()

In [ ]:
#this stuff takes forever
#df_entities = pd.DataFrame(columns=["id","hashtags","mentions","annotations"])
entities = list()
for tweet in tqdm(data["data"]):
    
    tweet_id = tweet["id"]
    
    try:
        hashtags = [hashtag["tag"].lower() for hashtag in tweet["entities"]["hashtags"]]
    except KeyError:
        hashtags = None
    
    try:
        mentions = [mention["username"].lower() for mention in tweet["entities"]["mentions"]]
    except KeyError:
        mentions = None
        
    try:
        annotations = [annotation["normalized_text"].lower() for annotation in tweet["entities"]["annotations"]]
    except KeyError:
        annotations = None
        
    entities.append([tweet_id,hashtags,mentions,annotations])
    
    #entities = pd.DataFrame([[tweet_id,hashtags,mentions, annotations]], columns=["id","hashtags","mentions", "annotations"])
    #df_entities = df_entities.append(entities, ignore_index=True)

In [ ]:
df_entities = pd.DataFrame(entities,columns=["id","hashtags","mentions","annotations"])
df_entities.head()

In [ ]:
# Merge entities into df_tweets
df_tweets_with_entities = pd.merge(left=df_tweets,
                                   right=df_entities,
                                   left_on="id",
                                   right_on="id",
                                   how="left")

In [ ]:
# Drop columns that are not needed
df_tweets_with_entities.drop(['entities.mentions', 'entities.hashtags', 'entities.urls','entities.annotations',
                              'in_reply_to_user_id', 'referenced_tweets', 'geo.place_id', 'geo.coordinates.type',
                              'geo.coordinates.coordinates', 'withheld.copyright', 'withheld.country_codes', 'entities.cashtags'],
                              axis=1, inplace=True)

In [ ]:
# Rename columns
df_tweets_with_entities = df_tweets_with_entities.rename(columns={'public_metrics.retweet_count': 'retweet_count',
                                                                  'public_metrics.reply_count': 'reply_count',
                                                                  'public_metrics.like_count': 'like_count',
                                                                  'public_metrics.quote_count': 'quote_count'})

In [ ]:
df_tweets_with_entities.head()

In [ ]:
df_tweets_with_entities.shape

In [ ]:
# Export file as csv
df_tweets_with_entities.to_csv(r"C:\Users\jawo19ad\Dropbox (CBS)\Master thesis data\df_tweets.csv")